# ConvLSTM model

Import libraries and modules.

In [ ]:
%cd ..
%cd ..
# move to the root directory of the git

In [ ]:
import importlib
import torch
import copy

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch.nn as nn

# Enable interactive widgets in Jupyter Notebook
%matplotlib widget

# import torch.nn.functional as F

from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models.ConvLSTM_model.ConvLSTM_pytorch.convlstm import ConvLSTM
from pre_processing.encode_decode_csv import decode_from_csv
from pre_processing.normalization import * 
from post_processing.cool_animation import plot_animation
from post_processing.plots import plot_losses

In [ ]:
# importlib.reload(convlstm)

Check if GPU is available.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

The following lines create variables to more easily specify what we use the model for (i.e., train and validate, test with dataset 1 and so on) in the following functions.

In [ ]:
train_val = 'train_val'
test1 = 'test1'
test2 = 'test2'
test3 = 'test3'

Load data from decoder.

**Please note**: 
\
inputs and targets in <code>train_val</code> are stored not in the original manner. 

index 0 = 1, index 1 = 10, index 11 = 2, index 12 = 20 etc.

In [ ]:
# training and validation dataset
train_dataset = decode_from_csv(train_val)

In [ ]:
# first position is the samples, second decides between inputs or targets, notice new shape of inputs
train_dataset[0][0].shape

In [ ]:
# test dataset 1
test_dataset1 = decode_from_csv(test1)

In [ ]:
# test dataset 2
test_dataset2 = decode_from_csv(test2)

In [ ]:
# test dataset 3
test_dataset3 = decode_from_csv(test3)

In [ ]:
# From FAT application
fig, axs = plt.subplots(1, 4, figsize=(10, 5))

# specify which entry of the dataset to plot
numb = 0

axs[0].imshow(train_dataset[numb][0][0][0].cpu(), cmap='terrain', origin='lower')
axs[0].set_title('DEM')

axs[1].imshow(train_dataset[numb][0][0][1].cpu(), cmap='RdBu', origin='lower')
axs[1].set_title('Slope X')

axs[2].imshow(train_dataset[numb][0][0][2].cpu(), cmap='RdBu', origin='lower')
axs[2].set_title('Slope Y')

axs[3].imshow(train_dataset[numb][0][0][3].cpu(), cmap='Set1', origin='lower')
axs[3].set_title('Water Depth time step 0')

In [ ]:
plot_animation(2, train_dataset, train_val)

In [ ]:
# Normalize the inputs and outputs using training dataset
scaler_x, scaler_y = scaler(train_dataset)

normalized_train_dataset = normalize_dataset(train_dataset, scaler_x, scaler_y)
normalized_test_dataset = normalize_dataset(test_dataset1, scaler_x, scaler_y)

In [ ]:
# Split dataset into train and validation
train_percnt = 0.8
train_size = int(train_percnt * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(normalized_train_dataset, [train_size, val_size])

In [ ]:
# Model, check, not sure num_layers is the expected amount of outputs
model = ConvLSTM(input_dim = train_dataset[0][0].shape[1], hidden_dim = 2, kernel_size = (3, 3), num_layers = 1,
         batch_first=True, bias=False, return_all_layers = True).to(device)
# return all layers has to be true to obtain all the outputs I think

In [ ]:
type(train_dataset[0][0])

In [ ]:
model

In [ ]:
def train_epoch(model, loader, optimizer, device):
    model.to(device)
    model.train() # specifies that the model is in training mode

    losses = []

    for batch in loader:
        x = batch[0].to(device)
        y = batch[1].to(device)

        # Model prediction
        preds, _ = model(x)
        list_preds = torch.cat(preds, dim=0)

        # MSE loss function
        loss = nn.MSELoss()(list_preds, y)
        
        losses.append(loss.cpu().detach())
        
        # Backpropagate and update weights
        loss.backward()   # compute the gradients using backpropagation
        optimizer.step()  # update the weights with the optimizer
        optimizer.zero_grad(set_to_none=True)   # reset the computed gradients

    losses = np.array(losses).mean()

    return losses

In [ ]:
def evaluation(model, loader, device):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode 

    losses = []

    for batch in loader:
        x = batch[0].to(device)
        y = batch[1].to(device)

        # Model prediction
        preds, _ = model(x)
        list_preds = torch.cat(preds, dim=0)

        # MSE loss function
        loss = nn.MSELoss()(list_preds, y)
        
        losses.append(loss.cpu().detach())
        
        # Backpropagate and update weights
        loss.backward()   # compute the gradients using backpropagation

    losses = np.array(losses).mean()

    return losses

In [ ]:
# Set training parameters
learning_rate = 0.001
batch_size = 16 # Only have 64 and 16 samples for training and validation, I think should be kept small
num_epochs = 400

# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Create the training and validation dataloaders to "feed" data to the model in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_losses = []
val_losses = []

for epoch in range(1, num_epochs+1):
    # Model training
    train_loss = train_epoch(model, train_loader, optimizer, device=device)

    # Model validation
    val_loss = evaluation(model, val_loader, device=device)

    if epoch == 1:
        best_loss = val_loss
    
    if val_loss<=best_loss:
        best_model = copy.deepcopy(model)
        best_loss = val_loss
        best_epoch = epoch

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch%10 == 0:
        print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                            "\t validation loss:", np.round(val_loss,4))
        
model = copy.deepcopy(best_model)

In [ ]:
plot_losses(train_losses, val_losses, 'ConvLSTM')